# Exploitation des LLM pour l'extraction d'information

- Utilisation d'un LLM depuis une interface python pour la stadardisation des chaines de traitement
- Exploitation rapide (sans fine-tuning)



<img src="ressources/chain.png"  width="600">

On aborde les problèmes séquentiellement en partant du LLM:

1. Faire tourner un LLM en python
1. Traiter un pdf ou un fichier texte
1. Coté LLM, contraindre les sorties et les analyser
1. Consuire une chaîne de traitements
1. Evaluer les performances

# 1. Faire tourner un LLM en python

Il existe plusieurs méthodes pour exploiter des LLM dans un programme python:
- méthode **locale** (le LLM tourne sur notre ordinateur/serveur)
- **API distante** (le LLM tourne chez Google, OpenAI ou autre)

## 1.1 Solution avec ollama

In [ ]:
# clé de l'API google: https://aistudio.google.com/app/u/1/apikey
cle_vguigue = # mettre la votre

In [ ]:
# !pip install ollama

In [ ]:
import ollama
from ollama import chat,generate
from ollama import ChatResponse

In [ ]:
# Initialize the language model
ans = ollama.generate(model='gemma3:270m', prompt='Why is the sky blue?')


In [ ]:
print(ans.response)

### Ne pas confondre la réponse (unique) et le chat (dialogue)

Note: generate ne répond qu'à une seule question. Pour passer à une logique de dialogue, il faut utiliser ```chat```

In [ ]:
messages = [{"role" : "user", "content": "Quelle est la différence entre un chat et un chien? Réponse très courte"}] # on part sur une liste avec des acteurs
dialog = ollama.chat(model='llama3.2', messages=messages)

print(dialog["message"]["content"])

In [ ]:
# ajout des éléments de dialogue puis relance
messages.append({"role": "assistant", "content": dialog["message"]["content"]})
messages.append({"role": "user", "content": "Et lequel est plus indépendant ?"})

dialog = ollama.chat(model='llama3.2', messages=messages)

print(dialog["message"]["content"])

Ne pas hésiter à afficher la liste `messages` pour bien comprendre ce qui se passe dans le dialogue (l'ensemble de la conversation est bien redonnée à chaque fois au modèle de langue)

In [ ]:
# bac à sable


### Jouer avec la température

Les modèles de langue sont liés à des algorithmes de post-processing, le plus connu étant beam-search: 
- les sorties sur le prochain mot correspondent à une distribution de probabilité
- il est possible de faire des tirages plutôt que de prendre le mot le plus probable
- plus la température est faible, plus on prend le mot le plus vraisemblable, plus elle est faible, plus on s'autorise de l'exploration

In [ ]:
custom_options = {"temperature": 0.}

ans = ollama.generate(model='gemma3:270m', prompt='Why is the sky blue?', options=custom_options ) # deterministe
print(ans.response)

In [ ]:
custom_options = {"temperature": 0.9}
ans1 = ollama.generate(model='gemma3:270m', prompt='Why is the sky blue?', options=custom_options ) # stochastique ++
ans2 = ollama.generate(model='gemma3:270m', prompt='Why is the sky blue?', options=custom_options ) # 
print(ans1.response)
print("---------------------")
print(ans2.response)

## 1.2 Envisager d'autres API

### Google
**Il est possible de tester gratuitement les API**

Accès aux outils de **google**: [lien](https://aistudio.google.com/app/u/1/apikey?hl=fr&pli=1)
- Créer une clé sur la page précédente (+créer un nouveau projet)<BR>
Avec les limites suivantes [lien](https://ai.google.dev/gemini-api/docs/rate-limits?hl=fr)

### openAI
**Il N'est PAS possible de tester gratuitement les API**

Guide d'accès: [lien](https://platform.openai.com/docs/overview)


#### Google

In [ ]:
# pour l'installation
# !pip install -q -U google-genai

In [ ]:
from google import genai

client = genai.Client(api_key=cle_vguigue)

response = client.models.generate_content(
    # attention, vous n'avez pas les mêmes droits avec les différents modèles
    # gemini-2.5-flash-lite | gemini-2.0-flash | gemini-2.5-pro 
    model="gemini-2.5-flash-lite", contents="Explain how AI works in a few words"
)
print(response.text)

Il est évidemment possible de régler la température sur ce type de modèle... Et il est possible de jouer de la même manière avec OpenAI, Anthropic ou Perplexity (mais seul Google donne accès à une version de démo gratuite)

## 1.3 Passer par Huggingface

ATTENTION: on utilisera Huggingface pour 2 choses assez différentes:
1. Des modèles génératifs disponibles pour faire les opérations de ce TP
1. Des modèles "encoders" pour le TP suivant 

On peut choisir les modèles parmi la (longue) liste: [lien](https://huggingface.co/models)
- Attention à la taille, les ressources deviennent vite conséquantes pour le fonctionnement
- privilégier les modèles *à la mode* dans un premier temps (qwen, llama, gemma, phi, ...)


In [ ]:
# ! pip install transformers, torch
# !pip install bitsandbytes
### il faut une version ancienne de numpy!
# ! pip install numpy==1.26.3

In [ ]:
import torch, numpy
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from transformers import pipeline

import re
print(transformers.__version__)

# Vérifier si le GPU est disponible
# pour les PC
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# pour les mac
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:

# on prend un petit modèle pour limiter les calculs
model_name = "Qwen/Qwen3Guard-Gen-0.6B"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    dtype="auto",
    device_map=device,
    # quantization_config=quant_config,
    torch_dtype=torch.float32, # sur mac, seul le 32 fonctionne, sur PC, on peut descendre à 16
)


In [ ]:

# Charger un modèle génératif pré-entraîné 
generator = pipeline("text-generation", model=model, tokenizer =tokenizer)

# Donner un prompt
prompt = "Once upon a time,"
outputs = generator(prompt, max_new_tokens=100)

# Afficher le résultat
print(outputs[0]["generated_text"])

## 2. Charger des fichiers de nature différentes (textes, pdf, doc, ...)

- Charger un document dans la chaine, que ce soit un texte ou un pdf

In [ ]:
# installation du module
# !pip install langchain_community
# !pip install docx2txt

In [ ]:
# on passe déjà par langchain... On verra la suite des possibilités un peu plus tard
from langchain_community.document_loaders import DirectoryLoader, TextLoader, PyPDFLoader, Docx2txtLoader, UnstructuredFileLoader
from langchain_community.document_loaders import UnstructuredFileLoader

In [ ]:

path_to_data = "./ressources/docs"

# Pour charger des fichiers TXT uniquement
txt_loader = DirectoryLoader(
    path=path_to_data,               # Ton répertoire
    glob="**/*.txt",             # Motif pour les fichiers TXT
    loader_cls=TextLoader,       # Loader utilisé
    show_progress=True
)

# Pour charger des fichiers PDF uniquement
pdf_loader = DirectoryLoader(
    path=path_to_data,
    glob="**/*.pdf",
    loader_cls=PyPDFLoader,
    show_progress=True
)

docx_loader = DirectoryLoader(
    path=path_to_data,
    glob="**/*.docx",
    loader_cls=Docx2txtLoader
)

# DOC
doc_loader = DirectoryLoader(
    path=path_to_data,
    glob="**/*.doc",
    loader_cls=UnstructuredFileLoader
)

# Charger les deux types de documents
txt_docs = txt_loader.load()
pdf_docs = pdf_loader.load()
docx_docs = docx_loader.load()
doc_docs = doc_loader.load() # note: il n'y en a pas dans le répertoire

In [ ]:

# Fusionner les documents
all_docs = txt_docs + pdf_docs + docx_docs + doc_docs

print(f"Nombre total de documents chargés : {len(all_docs)}")
print(all_docs[0].page_content[:500])  # aperçu du premier document


N'hésitez pas à jouer avec all_docs et à faire le lien avec les documents effectivement présent dans le répertoire `ressources/docs`.

In [ ]:
# pour vos tests

# 3. Analyser un texte avec un LLM

On va travailler avec ollama et HuggingFace pour tenter d'analyser un texte

1. Construction d'un prompt
1. Passage du prompt et du texte à analyser au LLM
1. Optimiser / jouer avec le prompt
1. Contrainte sur la réponse

Note: on va construire la chaine avec un petit modèle de langue, pour ne pas perdre inutilement de temps et d'énergie... Ce modèle n'est pas le plus performant!

In [ ]:
# copie de la cellule initiale = chargement de ollama

# import ollama
# from ollama import chat,generate
# from ollama import ChatResponse

## 3.1 Construction du prompt

Demander au modèle de langue de récupérer les personnes, les lieux, les organisations et les dates dans un texte


In [ ]:
# question préliminaire:
# commençons par visualiser le premier texte du corpus
txt = all_docs[0].page_content
print(txt)
# il est possible de récupérer d'autres informations que le contenu (e.g. le nom de fichier)
source = all_docs[0].metadata['source']
print(source)

In [ ]:
# prompt naïf
prompt = "Trouver les personnes, les lieux, les organisations et les dates dans le texte: "

to_analyse = prompt + txt

ans = ollama.generate(model='gemma3:270m', prompt=to_analyse)
print(ans.response)

## 3.2 Optimisation du prompt

Proposition de construction interactive: on va utiliser chatgpt (ou le modèle de votre choix) pour tenter de construire un prompt efficace. Je vous propose le texte suivant (modifiable à souhait):

```
Je veux construire un prompt pour analyser des textes. Mon but est d'extraire les personnes, les lieux, les organisations et les dates. Peux tu me faire une proposition?
```

Note: dans mes essais, le chatbot anticipe et propose dès le début de formatter les réponses...
Note 2: la proposition de la boite ci-dessous correspond à la réponse de chatgpt


In [ ]:
# Construction d'un prompt 
prompt = """
Rôle : Tu es un système d’analyse de texte spécialisé dans l’extraction d’entités nommées.
Tâche :
Analyse le texte suivant et identifie les entités mentionnées. Classe-les dans les catégories suivantes :
Personnes : noms de personnes individuelles.
Lieux : pays, villes, régions, adresses, lieux géographiques.
Organisations : entreprises, institutions, associations, administrations, etc.
Dates : jours précis, mois, années, périodes temporelles.

Format de sortie attendu (JSON clair et structuré) :
{
  "Personnes": ["..."],
  "Lieux": ["..."],
  "Organisations": ["..."],
  "Dates": ["..."]
}

Texte à analyser :"""

In [ ]:

to_analyse = prompt + txt

ans = ollama.generate(model='gemma3:270m', prompt=to_analyse)
print(ans.response)

## 3.3 Passage à un modèle plus performant (via ollama ou les API google)


In [ ]:
ans = ollama.generate(model='llama3.2', prompt=to_analyse)
print(ans.response)

Impact du prompt: seuls les noms propres sont considérés comme des personnes. 

Si on modifie:
`Personnes : noms de personnes individuelles.`<BR>
en: `Personnes : mentions, nom, désignation de personnes`

Pour améliorer les performances, il est possible de donner des exemples pour les différentes catégories... Vous pouvez les inventer ou en demander à votre chatbot favori.

Comment forcer le LLM à ne sortir qu'un JSON mais pas d'explications? <BR>
Note: n'hésitez pas à demander à un chatbot

In [ ]:
# TODO : jouer avec le prompt

### Avec une API Google

In [ ]:
from google import genai

client = genai.Client(api_key=cle_vguigue)

response = client.models.generate_content(
    # Gemini 2.5 Flash-Lite | Gemini 2.0 Flash | Gemini 2.5 Pro
    model="gemini-2.5-flash-lite", contents=to_analyse
)
print(response.text)

# 4. Construire une chaîne de traitement

Introduction de `langchain` (déjà croisé pour l'importation des documents) pour la construction d'une chaine générique où les outils sont interchangeable

1. Charge tous les documents d'un répertoire (en txt ou pdf) 
2. Passe chaque document dans un llm avec un prompt fixé pour obtenir un json 
3. Vérifie que le JSON est bien formaté
4. Changer de modèle de langue pour montrer l'intérêt de la chaîne

On va créer une fonction pour chaque étape et construire la chaîne

In [ ]:
# ! pip install langchain_ollama 

In [ ]:
import os
import re
import json
from langchain_community.document_loaders import TextLoader, PyPDFLoader
from langchain.prompts import PromptTemplate
from langchain_core.runnables.base import RunnableSequence
from langchain.schema import BaseOutputParser
from langchain_ollama.llms import OllamaLLM

## 4.1 Construction d'une chaîne

In [ ]:

# -------- 1. Charger tous les documents --------
# l'approche est plus élégante que tout à l'heure: on s'interroge sur les types de fichier au fur et à mesure

def load_documents_from_directory(directory_path):
    docs = []
    for filename in os.listdir(directory_path):
        file_path = os.path.join(directory_path, filename)
        if filename.endswith(".txt"):
            loader = TextLoader(file_path, encoding="utf-8")
            docs.extend(loader.load())
        elif filename.endswith(".pdf"):
            loader = PyPDFLoader(file_path)
            docs.extend(loader.load())
    return docs


In [ ]:
# -------- 2. Définir le LLM + Prompt --------

# Construction d'un prompt [copie de la cellule définie dans la section précédente]
# ATTENTION => Ajout du document dans le prompt + doublement des autres accolades

prompt = """
Rôle : Tu es un système d’analyse de texte spécialisé dans l’extraction d’entités nommées.
Tâche :
Analyse le texte suivant et identifie les entités mentionnées. Classe-les dans les catégories suivantes :
Personnes : noms de personnes individuelles.
Lieux : pays, villes, régions, adresses, lieux géographiques.
Organisations : entreprises, institutions, associations, administrations, etc.
Dates : jours précis, mois, années, périodes temporelles.

Format de sortie attendu (JSON clair et structuré) :
{{
  "Personnes": ["..."],
  "Lieux": ["..."],
  "Organisations": ["..."],
  "Dates": ["..."]
}}

Texte à analyser :
{document}"""

In [ ]:
# LLM et prompt
# Note: le LLM est dans un objet Langchain (astuce pour les rendre interchangeable => cf question suivante)

llm = OllamaLLM(model="llama3.2")
request = PromptTemplate.from_template(prompt)


In [ ]:

# -------- 3. Validation et récupération du JSON --------
# Parser JSON robuste
class SafeJsonOutputParser(BaseOutputParser):
    def parse(self, text: str):
        try:
            return json.loads(text)
        except json.JSONDecodeError:
            # print("ERR: ", text)
            # tentative de réparation si l'output n'est pas un JSON strict
            match = re.search(r"\{(.*)\}", text, re.DOTALL)
            text = "{"+match.group(1)+"}" # se limiter à ce qui se trouve dans les accolades
            return json.loads(text)

json_parser = SafeJsonOutputParser()


In [ ]:
# -------- 4. Pipeline complet --------

chain = request | llm | json_parser


In [ ]:

def process_documents(directory_path):
    docs = load_documents_from_directory(directory_path)
    results = dict()
    for doc in docs:
        print(doc.metadata)
        try:
            res = chain.invoke({"document": doc.page_content})
            results[doc.metadata['source']] = res
        except Exception as e:
            print(f"Erreur avec {doc.metadata}: {e}")
    return results
    
# calcul sur l'ensemble des documents
output = process_documents("./ressources/docs")


In [ ]:
# vérification des sorties

print(output)

## 4.2 Changer de modèle de langue, en conservant la chaîne

Passage à gemini

In [ ]:
# ! pip install langchain-google-genai

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

# Crée un LLM basé sur Gemini
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite", api_key=cle_vguigue)


In [ ]:
# CODE INCHANGE
# -------- 4. Pipeline complet --------

chain = request | llm | json_parser


In [ ]:
# CODE INCHANGE

def process_documents(directory_path):
    docs = load_documents_from_directory(directory_path)
    results = dict()
    for doc in docs:
        print(doc.metadata)
        try:
            res = chain.invoke({"document": doc.page_content})
            results[doc.metadata['source']] = res
        except Exception as e:
            print(f"Erreur avec {doc.metadata}: {e}")
    return results
    
# calcul sur l'ensemble des documents
output = process_documents("./ressources/docs")


In [ ]:
print(output)

# 5. Evaluation des performances

On dispose d'une vérité terrain (toute relative) dans le fichier `ressources/verite_terrain.pkl`. Nous allons le charger et comparer les contenus, ce qui n'a rien de trivial!

1. Comparer seulement les résultats pour lesquels nous avons réussi l'extraction
1. Comparer les listes par types
1. [OPT] Introduire une distance seuil pour savoir si les entités sont les mêmes ou pas (e.g. la Manche *vs* Manche)

In [ ]:
# Récupération du fichier

import pickle as pkl

with open("ressources/verite_terrain.pkl", "rb") as f:
    gt = pkl.load(f)

# le formatage du fichier est le suivant (et le même que celui de la chaine ci-dessus):
# gt[nom_du_fichier] = {"Personnes":[...], "Lieux": ...}

## 5.1 Version basique en "exact match"

Dans un premier temps, on calcule le pourcentage de matching par classe.

Il faudrait ensuite  calculer pour chaque classe la précision et le rappel.

In [ ]:
import numpy as np

res = None

# parcours de tous les documents
for k in gt.keys():
    y = gt[k]

    if res == None:
        res = dict()
        for ke in y:
            res[ke] = 0
            res[ke+"-tot"] = 0

    
    try:
        yhat = output['./'+k]
        # parcours des types d'entites
        for ke in y.keys():
            # TP
            # print(y[ke], yhat[ke])
            nb_commun = np.intersect1d(y[ke], yhat[ke])
            res[ke] += len(nb_commun)
            res[ke+"-tot"] += len(y[ke]) 
    except Exception as e:
        # print(k)
        for ke in y:
            res[ke+"-tot"] += len(y[ke]) 

print(res)

In [ ]:
# pourcentage de reconnaissance par classe:

for k in res.keys():
    if "-tot" in k: continue
    print(k, res[k]/res[k+'-tot'])

## 5.2 Vers une version plus robuste

Introduction de la distance d'édition
1. Comprendre la distance de Levenstein
1. La Comparaison devient bien plus difficile: il faut mesurer toutes les distances et seuiller!

In [ ]:
# ! pip install python-Levenshtein

In [ ]:

import Levenshtein

mot1 = "chat"
mot2 = "chats"
mot3 = "chien"

# Calcul de la distance d'édition
dist1 = Levenshtein.distance(mot1, mot2)
dist2 = Levenshtein.distance(mot1, mot3)

print(f"Distance entre '{mot1}' et '{mot2}' : {dist1}")
print(f"Distance entre '{mot1}' et '{mot3}' : {dist2}")

# 6. Pour aller plus loin

Adapter la chaine de traitement pour extraire les entités mais aussi les relations entre ces entités
